In [ ]:
import pandas as pd
import os.path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('debug/monitoring.csv', sep='\t')
keys_as_float = ['tmp', 'val', 'config', 'sb_oc', 'sb_unused']
for key in keys_as_float: dataset[key] = dataset[key].apply(lambda x : None if x == 'None' else float(x))

In [ ]:
dataset.head()

# Investigate global usage

In [ ]:
overall = dataset.loc[dataset['rec'] == 'global']
overall_cpu = overall.loc[overall['res'] == 'cpu']
overall_mem = overall.loc[overall['rec'] == 'mem']

In [ ]:
# Convert percent to CPU cores
pd.options.mode.chained_assignment = None  # default='warn'
overall_cpu['cores'] = overall_cpu['val'] * overall_cpu['config']

In [ ]:
palette = sns.color_palette("Set2", overall_cpu['res'].nunique())

g_val = sns.lineplot(data=overall_cpu, x='tmp', y='cores', hue='res', palette=palette)

# Investigate CPU usage

In [ ]:
subsets = dataset.loc[dataset['rec'] == 'subset']
subsets_cpu = subsets.loc[subsets['res'] == 'cpu']
subsets_mem = subsets.loc[subsets['rec'] == 'mem']

In [ ]:
# Convert percent to CPU cores
pd.options.mode.chained_assignment = None  # default='warn'
subsets_cpu['cores'] = subsets_cpu['val'] * subsets_cpu['config']

In [ ]:
palette = sns.color_palette("Set2", subsets_cpu['subset'].nunique())

g_val = sns.lineplot(data=subsets_cpu, x='tmp', y='val', hue='subset', linestyle ='--', palette=palette, legend=False)
g_config = sns.lineplot(data=subsets_cpu, x='tmp', y='config', hue='subset', palette=palette)

In [ ]:
import json
subsets_unarchived = subsets
example = json.loads(subsets_unarchived['sb_dsc'].iloc[0])
print(len(example['consumer_list']))

In [ ]:
subsets_unarchived['list_vm']  = subsets_unarchived['sb_dsc'].apply(lambda x : json.loads(x)['consumer_list'])
subsets_unarchived['count_vm'] = subsets_unarchived['sb_dsc'].apply(lambda x : len(json.loads(x)['consumer_list']))

In [ ]:
palette = sns.color_palette("Set2", subsets_unarchived['res'].nunique())

g_val = sns.lineplot(data=subsets_unarchived.loc[subsets_unarchived['res'] == 'cpu'], x='tmp', y='count_vm', hue='res', palette=palette)
g_val = sns.lineplot(data=subsets_unarchived.loc[subsets_unarchived['res'] == 'mem'], x='tmp', y='count_vm', hue='res', linestyle='--')

# Investigate VM usage

In [ ]:
vm = dataset.loc[dataset['rec'] == 'vm']
vm_cpu = vm.loc[vm['res'] == 'cpu']
vm_mem = vm.loc[vm['rec'] == 'mem']

In [ ]:
palette = sns.color_palette("Set2", vm_cpu['subset'].nunique())

g_val = sns.lineplot(data=vm_cpu, x='tmp', y='val', hue='subset', linestyle ='--', palette=palette, legend=False)
g_config = sns.lineplot(data=vm_cpu, x='tmp', y='config', hue='subset', palette=palette)

# Dive into subset configuration